# Section Project:

For the final project for this section, you're going to train a DP model using this PATE method on the MNIST dataset, provided below.

## Import Modules

In [81]:
import random
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import Dataset, DataLoader

from syft.frameworks.torch.differential_privacy.pate import perform_analysis

## Prepare Data

### Load the MNIST Training & Test Datasets

In [78]:
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=None)
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=None)

print("Training Set Size:", len(mnist_trainset))
print("Test Set Size:", len(mnist_testset))
print()
print("Min Data Value:", torch.min(mnist_trainset.data.min(), mnist_testset.data.min()))
print("Max Data Value:", torch.max(mnist_trainset.data.max(), mnist_testset.data.max()))
print()
print("Train Label Counts:", {label.item():count.item() for label, count in zip(*torch.unique(mnist_trainset.targets, return_counts=True))})
print("Test Label Counts:", {label.item():count.item() for label, count in zip(*torch.unique(mnist_testset.targets, return_counts=True))})

Training Set Size: 60000
Test Set Size: 10000

Min Data Value: tensor(0, dtype=torch.uint8)
Max Data Value: tensor(255, dtype=torch.uint8)

Train Label Counts: {0: 5923, 1: 6742, 2: 5958, 3: 6131, 4: 5842, 5: 5421, 6: 5918, 7: 6265, 8: 5851, 9: 5949}
Test Label Counts: {0: 980, 1: 1135, 2: 1032, 3: 1010, 4: 982, 5: 892, 6: 958, 7: 1028, 8: 974, 9: 1009}


1. Prepare Data
    1. Split the training dataset into `n + 1` smaller datasets where `n` is the number of teacher models
    2. Define a Dataset class and a DataLoader that can give batches of data for all `n` teacher datasets
2. Define Model(s)
    1. A simple ConvNet for both the main model and all teacher models
    2. If too slow: custom `nn.Module` that can process `n` batches at once for all `n` teachers
3. Train Teachers
4. Label Unlabeled Training Dataset in a Differentially Private Manner
    1. Generate raw labels
    2. PATE analysis to find a proper `epsilon` value
    3. Add proper noise to the label counts
    4. Take the labels with most counts
5. Train the Main Model On the Training Dataset with Generated Labels
6. Test on the Test Dataset